<div style="float:left;margin:5px 10px 5px 10px" markdown="1">
    <img src="images/auc.png" width="300">
</div>

<div style="float:right;margin-top:10px" markdown="1">
    <h3><i>Text Mining & Collective Intelligence</i></h3>
</div>

<br><br><br><br>

<center><h1>Making Recommendations</h1>

<br>

<h3>by Gianluca E. Lebani</h3>
<h4>• 03 Nov. 2017 •</h4>

</center>

<br>

>### Today
>
>- [the MovieLens 1M Dataset](#the-MovieLens-1M-Dataset)
>
>
>- [Similarity Measures](#Similarity-Measures)
>
>
>- [user-to-user kNN](#user-to-user-kNN)
>
>
>- [item-to-item kNN](#item-to-item-kNN)

---

In [1]:
from __future__ import division

import math

from itertools import product, combinations
from operator import itemgetter
from collections import defaultdict

import numpy as np

from scipy.spatial import distance
from sklearn.metrics import mean_absolute_error

## the MovieLens 1M Dataset

The MovieLens 1M dataset has been developed by the members of the [GroupLens](https://grouplens.org/) lab in the Department of Computer Science and Engineering at the University of Minnesota.

The MovieLens 1M dataset in brief:

- Ratings: 1 million
- Users: 6040
- Rated Movies: 3592
- Rated Scale: {1, ... , 5}
- Additional information on the users: gender, age range, occupation, zip-code
- Additional infomation on the movies: genre

(A zipped version of this dataset should be available in the `./data` folder, in case not [download it](https://grouplens.org/datasets/movielens/1m/) and unzip it)

The dataset is composed by three files:

- `movies.dat`, providing information about the rated movies 
    - it follows the format `MovieID::Title::Genres`
    

- `users.dat`, providing information about the users
    - it follows the format `UserID::Gender::Age::Occupation::Zip-code`
    - each user has at least 20 ratings


- `ratings.dat`, encoding the ratings
    - it follows the format `UserID::MovieID::Rating::Timestamp`

In [2]:
id2movie = dict()

with open("data/ml-1m/movies.dat", "rb") as infile:
    for line in infile:
        movieId, movie, _ = line.split("::")
        id2movie[int(movieId)] = movie

In [3]:
user2movies_ratings = defaultdict(dict)

with open("data/ml-1m/ratings.dat", "rb") as infile:
    for line in infile:
        userId, movieId, rating = [int(el) for el in line.split("::")[:3]]
        user2movies_ratings[userId][movieId] = rating

---

<div style="float:left;margin:0 25px 10px 20px">
    <img src="images/your_turn.jpg" width="110">
</div>

#### Your Turn.

Explore the dataset:

- what is the average rating?


- which are the top-rated movies? And which are the lowest-rated ones?


- what are the average ratings for men and women?


- which movies received the highest rates from men? Which ones from women?


In [4]:
# your code here
for user, subdicteroy in user2movies_ratings_iteritems():
    

TypeError: unhashable type

---

### Use Case

In the following exercise we will use the **k Nearest Neighbors algorithm** (kNN)

- the most popular algorithm in the first RSs


- still the most-used algorithm for the collaborative filtering RSs


- conceptually simple and easy to implement


- it generally produces good predictions

#### TODAY'S EXERCISE: 

- to model the ratings given by subject  `4447`...
    - he gave 982 ratings


- ... for the following movies:

    - `Back to the Future (1985)`: id = `1270`
    - `Silence of the Lambs, The (1991)` : id = `593`
    - `Raiders of the Lost Ark (1981)` : id = `1198`

In [ ]:
target_movies = [593, 1198, 1270]

In [ ]:
# let's remove our target ratings from the dataset

target_ratings = dict([(i, user2movies_ratings[4447].pop(i)) for i in target_movies])

---

## Similarity Measures

Similarity/Distance Measures are used in many applications (e.g. clustering, classification, word modeling...) to measure how two objects are similar/different


- Depending on the applications, datapoints can be web pages, movies, users, words, concepts...


- Distances/Similarities are described by a single scalar, whose value is dependent on three factors:
    - the properties of the **objects** 
    - the **coding** scheme choose to describe the objects
    - the properites of the **measure**

In order to be qualifies as a **METRIC**, as distance/similarity measure must satisfy the following four conditions:

- the distance between any points is always **nonegative**


- the distance between two points equals $0$ iif two objects are **identical**


- it is **symmetric**, i.e. $d(x,y) = d(y,x)$


- **triangle inequality** must be satisfied, i.e. $d(x, z) \leq d(x, y) + d(y, z)$

The kNN algorithm is traditionally based on the use of traditional **statistical similarity measures**, where SMs are used to estimate the similarity between users (user-to-user kNN) or between items (item-to-item kNN).

In the years, many RS-tailored measures have been proposed. In most cases, the use of these SMs lead to superior performances, even if:

- some of these measures need information that is available only in certain architectures (e.g. social information)


- some of them have not been extensively tested (and understood)

### (Some) Statistical Similarity Measures

- The traditional similarity measures, widely used also outside of the RS literature (e.g. clustering, ML, retrieving applications, DSMs...)

**EUCLIDEAN SIMILARITY**

- euclidean distance is the ordinary straight-line distance between two points


- distance is calculated by using the Pythagorean theorem


- distance can be transformed into a similarity measure by summing 1 to the distance and inverting it

$$sim_{\ euclidean} (p,q)=\frac{1}{dist(p,q)} = \frac{1}{1 + \sqrt{\sum_{i = 1}^n (p_i - q_i)^2}}$$

In [ ]:
def euclidean_similarity(p, q):
    dist = math.sqrt(sum((pi-qi)**2 for pi,qi in zip(p, q)))
    sim = 1 / (1+dist)
    return sim    

**MANHATTAN SIMILARITY**

- the Manhattan (a.k.a. taxicab, cityblock, rectilinear, $L_1$) distance between two points is calculated as the sum of the absolute distances of their Cartesian coordinates


- similar to euclidean distance, but less influenced by outliers (i.e. by unusual values)

    - in practice, you obtain similar results most of the time

$$sim_{\ manhattan} (p,q) =\frac{1}{dist(p,q)} = \frac{1}{1 + \sum_{i = 1}^n |p_i - q_i|}$$

---

<div style="float:left;margin:0 25px 10px 20px">
    <img src="images/your_turn.jpg" width="110">
</div>

#### Your Turn.

Write a function called `manhattan_similarity()` that calculates this metric

In [ ]:
# your code here

def euclidean_similarity(p, q):
    dist = sum(abs(pi-qi) for pi,qi in zip(p, q))
    sim = 1 / (1+dist)
    return sim    

---

**COSINE SIMILARITY**

- the normalized dot product of two vectors, i.e. the cosine of the angle between them


- **scale-invariance**: cosine similarity is independent of vector length 
    - you can multiply your vector for any non-zero constant and the angle won't change
    - it should be used when vector length is irrelevant
    - in kNN, this means that it is independent of the absolute number of rating per user/item

$$sim_{\ cosine} (\vec{p},\vec{q})=\frac{\vec{p} \cdot \vec{q}}{\lVert \vec{p} \rVert \lVert \vec{q} \rVert} = \frac{\sum_{i = 1}^n p_i q_i}{\sqrt{\sum_{i = 1}^n (p_i)^2} \sqrt{\sum_{i = 1}^n (q_i)^2}}$$

In [ ]:
def cosine_similarity(p,q):
    d = sum(pi * qi for pi,qi in zip(p, q))
    mag_p = math.sqrt(sum([pi**2 for pi in p]))
    mag_q = math.sqrt(sum([qi**2 for qi in q]))
    sim = d / ( mag_p * mag_q)
    return sim

**PEARSON CORRELATION**: 


- measure of the strength of linear dependence between two variables


- calculated as the ratio between the variance that is shared between the two variables (covariance) and the product of their standard deviations (i.e. of their variance)

$$r(p,q) = \frac{\sum_{i = 1}^n (p_i - \bar{p})(q_i - \bar{q})}{\sqrt{\sum_{i = 1}^n (p_i - \bar{p})^2}\ \sqrt{\sum_{i = 1}^n (q_i - \bar{q})^2}}$$

- the correlation value between two variables is invariant to **scale** and **location** tranformations:
    - that is, if $a$, $b$, $c$, $d$ are constants and $a > 0$ and $c > 0$:

$$r(p,q) = r(a\cdot p + b, c\cdot q + d)$$

In [ ]:
def pearson_correlation(p,q):
    # this code does not scale properly. In the following example we rely on 
    # scipy.spatial.distance.correlation() to compute long vectors
    if len(p) > 99:
        return 1 - distance.correlation(p,q)        
    
    p_mean = sum(p) / len(p)
    p_deviations = [(pi-p_mean) for pi in p]
    
    q_mean = sum(q) / len(q)
    q_deviations = [(qi-q_mean) for qi in q]
    
    cov = sum(pd * qd for pd,qd in zip(p_deviations, q_deviations))
        
    sds_product = math.sqrt(sum((pd)**2 for pd in p_deviations) * sum((qd)**2 for qd in q_deviations))
    
    if sds_product != 0:
        r = cov / sds_product
    else:
        r = 0
    return r

**Pearson correlation** and **cosine similarity**:

- the pearson correlation is equivalent to the cosine similarity in which the values of the vectors has been **normalized to their arithmetic means**:

$$sim_{\ cosine} (p - \bar{p}, q - \bar{q}) = \frac{\sum_{i = 1}^n (p_i - \bar{p})(q_i - \bar{q})}{\sqrt{\sum_{i = 1}^n (p_i - \bar{p})^2}\ \sqrt{\sum_{i = 1}^n (q_i - \bar{q})^2}} = r(p,q) $$

- that is, the Pearson correlation and the cosine similarity are equivalent when the two vectors have **means of 0**

    - geometrically, the pearson correlation can be seen as the angle between the two vectors where the origin of the coordinate system is translated at the arithmetic mean values of the vectors
    - pearson correlation can be taught as as demeaned version of the cosine similarity

**JACCARD SIMILARITY** 

- a.k.a. **Tanimoto Coefficient**


- similarity is measured as the ratio of the cardinality of the intersection set (number of items that are in both sets) to that of the union set (number of items in either sets)


$$sim_{\ jaccard} (P, Q) = \frac{|P \cap Q|}{|P \cup Q|}$$

In [ ]:
def jaccard_sets(p,q): 
    intersection_cardinality = len(set(p).intersection(set(q)))
    union_cardinality = len(set(p).union(set(q)))
    sim = intersection_cardinality / union_cardinality
    return sim

- it is particularly useful  for **datasets with binary values** (e.g. 1s for a preference and 0s for indifference)
    - in these cases, you have two objects $P$ and $Q$ with $n$ attributes...
    - ... and you may use this measure to estimate the **overlap that $P$ and $Q$ share with their attributes**



- if $M_{11}$ represents the number of attributes where both objects have a value of 1 and $M_{10} + M_{01}$ represents the number of attributes where only one object has a value of 1:

$$sim_{\ jaccard} = \frac{M_{11}}{M_{01} + M_{10} + M_{11}}$$

In [ ]:
def jaccard_binary(p, q):
    """
    this is equivalent to the tanimoto() function by Segaran (2007:47) 
    """
    m_11, m_01, m_10 = 0, 0, 0
    for pi, qi in zip(p, q):

        if pi == 1:
            if qi == 1:
                m_11 += 1
            else:
                m_10 += 1
                
        elif qi == 1:
            m_01 += 1
    
    sim = m_11 / (m_10 + m_01 + m_11) 
    return sim

---

<div style="float:left;margin:0 25px 10px 20px">
    <img src="images/your_turn.jpg" width="110">
</div>

#### Your Turn.

Given the following binary vectors, find a way to use `jaccard_sets()` to obtain `jaccard_binary()`

In [ ]:
p = [1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1]
q = [1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0]

In [ ]:
 
jaccard_sets(p,q)

---

### Which one Should I use?

That really depends on your application:

- how are we modeling our problem (i.e. are we using binary features)?


- how do we want our measure to behave?
    - do we want to be less sensistive to outliers?
    - do we want it to be scale-invariat?
    - do we want it to be location-invariat?

Anyway, as a **rule of thumb**, the best practice is to try how the main similarity measures behave and decide on the basis of this empirical evaluation

---

## user-to-user kNN

Given an active user *a*:
- use  a similarity measure to determine the *k* most-similar users to *a*


- obtain the prediction on item *i* for user *a* by using one of the following aggregation approaches on the ratings from the neighborhood:
    - average
    - weighted sum
    - adjusted weighted aggregation (deviation-from-mean)
    
    
- choose the top-*n* items by selecting the *né items with the highest scores calculated by applying the previous steps on the items that haven’t been rated by the user *a*

### STEP 1: finding the top-50 similar users

Let's build the neighborhood for our target user `4447` by calculating his similarity with the other raters

In [ ]:
def calculate_similarity(ratings, id1, id2, measure, threshold = 0):
    # get the list of shared rated items
    shared = sorted(set(ratings[id1].keys()).intersection(set(ratings[id2].keys())))

    # ignore comparisons with too few overlapping ratings (default is 0)
    if len(shared) <= threshold:
        return 0
    
    sel_ratings = [np.asarray([v for (k,v) in ratings[i].items() if k in shared]) for i in [id1, id2]]
    
    # compute similarity
    sim = measure(*sel_ratings)
    return sim

In [ ]:
# let's calculate the similarities by using both the euclidean similarity and correlation (DURATION: 56 seconds)
measure2function = {"euclidean" : euclidean_similarity, "cosine": cosine_similarity, "correlation": pearson_correlation}

similarities = dict()
for measure, function in measure2function.items():
    similarities[measure] = dict()
    for id1, id2 in product([4447], user2movies_ratings.keys()):
        # do not compare our target user with himself
        if id1 == id2:
            continue
        similarities[measure][id2] = calculate_similarity(user2movies_ratings, id1, id2, function)

In [ ]:
# select the most similar users according to each measure
neighborhood = dict()
for measure in similarities.keys():
    neighborhood[measure] = dict(sorted(similarities[measure].iteritems(), key = itemgetter(1), reverse = True)[:50])
print neighborhood

### STEP 2: obtain the predictions for all the items of interest 

- in a real life scenario, we should obtain a predictions for all the items that were not rated by our user


- in this example, we will get predictions for just our three target movies

Let's use the following weighted score to aggregate our ratings:
 
- take the votes of all other critics and multiply them by their similarity with our target user


- sum these weihted votes for each item fo interest


- in order to handle the sparseness of the dataset (no movie has been rated by all the users), divide this score by the sum of all the similarities for critics that reviewed that movie

e.g. see the example from Segaran (2007: 15)

![alt text](images/weighting-users.png)

In [ ]:
def getPredictions(movieId, neighborhood, ratings):
    weigthed_scores = []
    similarities = []

    for user, sim in neighborhood.iteritems():
        if ratings[user].has_key(movieId):
            weigthed_scores.append(sim * ratings[user][movieId])
            similarities.append(sim)
    
    return sum(weigthed_scores) / sum(similarities)

In [ ]:
recommendations = defaultdict(dict)

for measure in similarities.keys():
    for movie in target_movies:
        recommendations[measure][movie] = getPredictions(movie, neighborhood[measure], user2movies_ratings)

### STEP 3: choose the top-items 

- in a real life scenario, you should choose the top-rated items and recommend them to the user


- in this exercise, we will compare the **rating predictions** produced by our RC against those produced by our user

is the **ranking** preserved?

In [ ]:
print "- original ratings:"

for movieID in target_ratings:
    print id2movie[movieID], "-->", target_ratings[movieID]

In [ ]:
for measure in similarities.keys():
    print "-", measure, "ratings:"
    for movieID in target_ratings:
        print id2movie[movieID], "-->", recommendations[measure][movieID]
    print

let's calculate the **Mean Absolute Error** 

- the difference between the real ratings and those produced by the RS


- i.e. if $\hat{y}_i$ is the predicted value of the $i$-th sample, $y_i$ is the true value and $n$ is the sample size:

$$MAE(y, \hat{y}) = \frac{1}{n} \sum_{i = 0}^{n - 1} \left| \ y_i, \hat{y_i}\ \right|$$

In [ ]:
true_ratings = [target_ratings[movieID] for movieID in target_ratings]

for measure in similarities.keys():
    print "-", measure, "MAE:",
    predicted = [recommendations[measure][movieID] for movieID in target_ratings]
    print mean_absolute_error(true_ratings, predicted)

---

<div style="float:left;margin:0 25px 10px 20px">
    <img src="images/your_turn.jpg" width="110">
</div>

#### Your Turn.

See what happens if:

- we specify a minimum rating overlap threshold (say, 10) in the `calculate_similarity()` function


- we change the size of the neighborhood (say, to 100)

In [ ]:
# your code here

---

## item-to-item kNN

Given an active user *a*:

- for each item in the database, use a similarity measure to determine its *k* most-similar items


- for each item *i* not rated by *a*, predict its rating on the basis of the *a*’s previous ratings of the items in the *i*'s neighborhood


- choose the top-*n* items by selecting the *n* items with the highest scores calculated in the previous step


### STEP 1: finding the top-25 similar items

re-arranging the ratings dictionary allows us to use the `calculate_distance()` function to calculate the item-based similarities as well

In [ ]:
# let's rearrange the dictionary of ratings 

movies2user_ratings = defaultdict(dict)

for user, user_ratings in user2movies_ratings.iteritems():
    for movie, rating in user_ratings.iteritems():
        movies2user_ratings[movie][user] = rating

To speed up the process, we will ignore all those movies that has not been rated by at least 1500 users

In [ ]:
filtered_movies2user_ratings = dict()
for movie in movies2user_ratings.keys():
    if len(movies2user_ratings[movie]) < 1500:
        continue
    else:
        filtered_movies2user_ratings[movie] = movies2user_ratings[movie]

print "-", len(movies2user_ratings) - len(filtered_movies2user_ratings), "movies have been descarded"
print "-", len(filtered_movies2user_ratings), "movies have been selected"

In [ ]:
# let's calculate the similarities by using only correlation (NOTE: this is very inefficient!)

similarities = defaultdict(dict)
for id1, id2 in combinations(filtered_movies2user_ratings.keys(), 2):
    similarities[id1][id2] = calculate_similarity(movies2user_ratings, id1, id2, pearson_correlation)

In [ ]:
# select the most similar items
neighborhood = dict()
for movie in similarities.keys():
    neighborhood[movie] = dict(sorted(similarities[movie].iteritems(), key = itemgetter(1), reverse = True)[:25])

---

<div style="float:left;margin:0 25px 10px 20px">
    <img src="images/your_turn.jpg" width="110">
</div>

#### Your Turn.

Choose a movie and explore its immediate neighborhood by printing its most similar movies and using MDS (multi Dimensional Scaling) to plot them in a 2-dimensional space

In [ ]:
# your code here

---

### STEP 2: obtain the predictions for all the items of interest 

For all the items that the user hasn't rated (here we restrict ourselves to our three target movies), the following weighted score is used to aggregate our ratings:

- for each pair of items composed by one item rated by our user and one of our items of interest, we calculate a score by multiplying their pairwise similarity and the rating for the known movie


- for each item of interest we sum all these scores


- the score is normalized by diving this total by the total of the pairwise similarity scores involving a item of interest

e.g. see the example from Segaran (2007: 24)

![alt text](images/weighting-item.png)

In [ ]:
def getPredictionsForItems(userId, neighborhood, ratings):
    weigthed_scores = []
    similarities = []

    for item, sim in neighborhood.iteritems():
        if ratings[item].has_key(userId):
            weigthed_scores.append(sim * ratings[item][userId])
            similarities.append(sim)
    
    return sum(weigthed_scores) / sum(similarities)

In [ ]:
recommendations = defaultdict(dict)

for movie in target_movies:
    recommendations[movie] = getPredictionsForItems(4447, neighborhood[movie], filtered_movies2user_ratings)

### STEP 3: choose the top-items 

- in a real life scenario, you should choose the top-rated items and recommend them to the user


- in this exercise, we will compare the **rating predictions** produced by our RC against those produced by our user

is the **ranking** preserved?

In [ ]:
print "- original ratings:"

for movieID in target_ratings:
    print id2movie[movieID], "-->", target_ratings[movieID]

In [ ]:
print "- predicted ratings:"
for movieID in target_ratings:
    print id2movie[movieID], "-->", recommendations[movieID]

let's calculate the **Mean Absolute Error** (i.e. the difference between the real ratings and those produced by the RS)

In [ ]:
true_ratings = [target_ratings[movieID] for movieID in target_ratings]
predicted = [recommendations[movieID] for movieID in target_ratings]

print "- MAE:", mean_absolute_error(true_ratings, predicted)

---

>#### For Next Tuesday:
>
> - Read **Sections 14.1, 14.2 and 14.4** of Zhai C. X. & S. Massung (2016) Text Data Management and Analysis: A Practical Introduction to Information Retrieval and Text Mining, Association for Computing Machinery and Morgan & Claypool
>
>
> - Read Blei, D. M. (2012) Probabilistic Topic Models. In *Communications of the ACM*, 55 (4), pp. 77 - 84
>
>
> - Prepare two questions about the topic of the lesson. Questions should be **sent by mail**, the subject line following the format: `“TMCI\_questions\_date-of-the-class\_[YOUR LAST NAME]”` (e.g. `TCMI\_questions_26/Sep\_Lebani`)
>

ignore what follows

In [ ]:
# Export this notebook as a HTML file
# !jupyter nbconvert TMCI-2017-w9a --output html_converted_notebooks/TMCI-2017-w9a